# Mission 13: 쇼핑몰 리뷰 감성 분석 (PEFT)

> **[중요] CUDA Error 발생 시 해결 방법**
>
> 만약 학습 도중 `CUDA error: device-side assert triggered` 오류가 발생한다면, 이는 주로 **라벨링 문제** 때문입니다.
> 이 오류는 한 번 발생하면 커널(Kernel) 상태가 꼬여서 코드를 수정해도 계속 같은 오류가 뜹니다.
> **반드시 상단 메뉴의 [Kernel] -> [Restart Kernel]을 눌러 커널을 재시작한 후 처음부터 다시 실행해주세요.**

## 1. 미션 소개
쇼핑몰 리뷰 데이터를 활용하여 감성(긍정/부정)을 분석하는 모델을 개발하는 미션입니다.
단순히 모델을 학습시키는 것을 넘어, **Full Fine-Tuning** 방식과 **PEFT (Parameter-Efficient Fine-Tuning)** 두 가지 방식을 모두 적용해보고, 그 효율성과 성능을 비교 분석하는 것이 핵심 목표입니다.

## 2. 미션 목표
1. **데이터 전처리**: Raw Json 데이터에서 텍스트와 라벨을 추출하고 학습 가능한 형태로 변환
2. **모델 학습 (2가지 방식)**:
    - **Full Fine-Tuning**: 모델의 모든 파라미터를 업데이트
    - **PEFT (LoRA 등)**: 일부 파라미터만 효율적으로 업데이트
3. **성능 및 효율성 비교**:
    - 학습 속도, 메모리 사용량, 모델 용량 비교
    - 정확도(Accuracy) 등 감성 분석 성능 비교
4. **결과 정리**: 코드와 마크다운을 통해 전체 프로세스와 결과 분석을 체계적으로 정리

## 3. 데이터 소개
- **데이터셋 경로**: `/mnt/nas/jayden_code/Codeit_Practice/Part3_mission_13/review-sentiment-analysis`
- **구조**:
    - 대분류: SNS / 쇼핑몰
    - 중분류: 패션, 화장품, 가전, IT기기, 생활
    - 소분류: (예: 여성의류, 남성의류 등)
    - 파일 형식: JSON
- **핵심 필드**:
    - `RawText`: 리뷰 텍스트 본문
    - `GeneralPolarity`: 감성 라벨 (-1: 부정, 0: 중립, 1: 긍정)

## 4. 데이터 분석 (EDA) 및 전처리 계획
1. **탐색적 데이터 분석 (EDA)**:
    - 카테고리별 데이터 분포 확인
    - 라벨(긍정/부정/중립) 불균형 확인
    - 리뷰 텍스트 길이 분포 분석
2. **전처리 (Preprocessing)**:
    - **라벨링 변환**: -1(부정), 0(중립), 1(긍정) -> 모델 학습용 클래스 (0, 1, 2)로 매핑
    - **데이터 통합**: 여러 JSON 파일을 pandas DataFrame으로 통합
    - **Train/Test 분리**: 학습 및 평가를 위한 데이터셋 분할 (예: 8:2)
    - **토크나이징**: Pre-trained 모델의 Tokenizer 적용

## 5. 진행 계획 (가이드라인 준수)
1. **기초 작업**: 데이터 탐색 및 Dataset 클래스 생성
2. **베이스라인 설정**: Hugging Face `transformers` 라이브러리의 Pre-trained 모델 선택 (예: `klue/roberta-base` or `beomi/KcELECTRA-base`)
3. **Full Fine-Tuning 구현**:
    - `Trainer` API 활용
    - 전체 파라미터 학습
    - 평가 지표(Accuracy, F1) 확인
4. **PEFT (LoRA) 구현**:
    - `peft` 라이브러리 활용
    - LoRA Config 설정 (rank, alpha 등)
    - 학습 진행 및 저장
5. **비교 및 리포트**:
    - 학습 시간, 저장 용량, 추론 성능 비교 테이블 작성
    - 결론 도출

---
*초보자 친화적인 코드로, 각 단계별로 주석을 상세히 달아 진행하겠습니다.*

## 1. 라이브러리 임포트
데이터 분석과 모델 학습에 필요한 라이브러리들을 불러옵니다.

In [1]:
# 필요한 라이브러리들을 불러옵니다.
# os: 파일 경로 탐색을 위해 사용
# json: JSON 형식의 데이터 파일을 읽기 위해 사용
# pandas: 데이터를 표 형태로 다루기 위해 사용 (엑셀과 비슷하다고 생각하시면 됩니다!)
# numpy: 수치 계산을 위해 사용
import os
import json
import pandas as pd
import numpy as np

# Hugging Face 라이브러리 (데이터셋 및 모델 관련)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score # 평가 지표

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


## 2. 데이터 로드 함수 정의
여러 폴더에 흩어져 있는 JSON 파일들을 재귀적으로 탐색하여 하나의 리스트로 모으는 함수입니다.

In [2]:
# 1. 데이터 로드 및 전처리 시작

# 데이터가 저장된 원본 경로입니다.
DATA_DIR = "/mnt/nas/jayden_code/Codeit_Practice/Part3_mission_13/review-sentiment-analysis"

# 모든 폴더를 돌면서 JSON 파일을 찾아 데이터를 모으는 함수를 만듭니다.
def load_data(root_dir):
    """
    지정된 경로(root_dir) 하위의 모든 폴더를 탐색하여 JSON 파일을 읽고,
    리뷰 텍스트(RawText)와 감성 라벨(GeneralPolarity)을 추출하여 리스트로 반환합니다.
    """
    all_data = [] # 데이터를 모을 빈 리스트 생성
    
    print("데이터 로드를 시작합니다... (시간이 조금 걸릴 수 있어요)")
    
    # os.walk를 사용하여 root_dir 하위의 모든 파일 경로를 방문합니다.
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".json"): # 파일 이름이 .json으로 끝나는 경우만 처리
                file_path = os.path.join(root, file)
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        
                        # JSON 구조 확인: 보통 'data' 리스트 안에 개별 리뷰가 들어있거나, 
                        # 파일 구조에 따라 다를 수 있으므로 리스트 형태인지 확인합니다.
                        # 데이터셋 명세를 보면 하나 혹은 여러개의 리뷰가 들어있을 수 있습니다.
                        
                        # 만약 data가 리스트라면 여러 리뷰가 들어있는 것입니다.
                        if isinstance(data, list):
                            for item in data:
                                # 필요한 데이터만 쏙쏙 뽑아냅니다.
                                if 'RawText' in item and 'GeneralPolarity' in item:
                                    text = item['RawText'] # 리뷰 텍스트
                                    # ★ 중요 수정: JSON의 라벨값이 문자열("1", "-1")로 되어있을 수 있으므로 int로 변환해줍니다.
                                    label = int(item['GeneralPolarity']) 
                                    
                                    # ★ 안전 장치: 라벨이 -1, 0, 1이 아닌 이상한 값이면 제외합니다.
                                    if label not in [-1, 0, 1]:
                                        continue

                                    # 리스트에 추가 (딕셔너리 형태)
                                    all_data.append({
                                        'text': text,
                                        'label': label
                                    })
                        # 만약 딕셔너리라면 하나의 객체일 수도 있습니다 (구조에 따라 다름, 일단 리스트라 가정하고 진행)
                        elif isinstance(data, dict):
                             # 혹시 data 자체가 하나의 리뷰 뭉치일 수도 있으니 구조에 맞춰 처리
                             # (실제 데이터 형태를 보고 수정이 필요할 수도 있습니다. 우선 일반적인 경우를 상정합니다)
                             if 'RawText' in data and 'GeneralPolarity' in data:
                                 label = int(data['GeneralPolarity'])
                                 if label not in [-1, 0, 1]:
                                     continue

                                 all_data.append({
                                     'text': data['RawText'],
                                     'label': label
                                 })
                                 
                except Exception as e:
                    print(f"파일 읽기 오류 발생 ({file}): {e}")
                    continue
                    
    print(f"총 {len(all_data)}개의 리뷰 데이터를 불러왔습니다!")
    return all_data

# 함수 실행!
raw_data = load_data(DATA_DIR)

데이터 로드를 시작합니다... (시간이 조금 걸릴 수 있어요)
총 184525개의 리뷰 데이터를 불러왔습니다!


## 3. 데이터 확인 (DataFrame 변환)
로드한 데이터를 Pandas DataFrame으로 변환하고, 데이터의 구조와 라벨 분포를 확인합니다.

In [3]:
# 모은 데이터를 보기 편하게 pandas DataFrame으로 변환합니다.
df = pd.DataFrame(raw_data)

# 상위 5개 데이터를 눈으로 확인해봅니다.
print("데이터 미리보기:")
display(df.head())

# 데이터 정보 확인 (빈 값이 있는지, 데이터 타입은 무엇인지)
print("\n데이터 정보:")
print(df.info())

# 라벨 분포 확인 (긍정, 부정, 중립 개수 확인)
# -1: 부정, 0: 중립, 1: 긍정
print("\n라벨 분포 확인:")
print(df['label'].value_counts())

데이터 미리보기:


,text,label
0,이것저것 바르기를 워낙 싫어하는 남편한테 최고의 제품이네요. 하나만 바르니 사용하기...,1
1,향은 남성 화장품 냄새가 좀 강해서 아쉽고 유통기한은 내년 3월까지라 무지 짧네요.,-1
2,가격 저렴하고 배송 빠르고 용량도 엄청나고 향도 나쁘지 않아요. 잘산 것 같아요.,1
3,1+1행사로 남편 선물했는데 향도 괜찮고 조금만 발라도 잘 발라진다고 좋은 거 같다...,1
4,향이 강하지 않고 깔끔한 느낌의 향이에요. 배송도 빨라서 좋았습니다. 다만 끈적임이...,0



데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184525 entries, 0 to 184524
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    184525 non-null  object
 1   label   184525 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.8+ MB
None

라벨 분포 확인:
label
 1    118870
 0     37592
-1     28063
Name: count, dtype: int64


## 4. 라벨 인코딩 (전처리)
모델 학습을 위해 라벨을 0부터 시작하는 정수형(0, 1, 2)으로 변환합니다.

In [4]:
# 라벨을 모델이 학습하기 편한 숫자(0, 1, 2)로 바꿔줍니다.
# Hugging Face 모델들은 보통 0부터 시작하는 정수 레이블을 기대합니다.
# -1 -> 0 (부정)
#  0 -> 1 (중립)
#  1 -> 2 (긍정)

label_map = {-1: 0, 0: 1, 1: 2}

# ★ 안전 장치: 이미 변환된 경우(0, 1, 2만 있는 경우)에는 다시 변환하지 않도록 체크합니다.
unique_labels = set(df['label'].unique())
print(f"현재 라벨 종류: {unique_labels}")

# 만약 -1이 포함되어 있다면 변환을 수행합니다.
if -1 in unique_labels:
    print("라벨 변환을 수행합니다...")
    df['label'] = df['label'].map(label_map)
else:
    print("이미 라벨이 변환되어 있거나, 변환할 필요가 없습니다.")

# 잘 바뀌었는지 확인
print("라벨 변환 결과 확인:")
print(df['label'].head())

# 변환된 라벨 분포도 다시 한 번 확인
print("\n변환된 라벨 분포:")
print(df['label'].value_counts())

현재 라벨 종류: {np.int64(0), np.int64(1), np.int64(-1)}
라벨 변환을 수행합니다...
라벨 변환 결과 확인:
0    2
1    0
2    2
3    2
4    1
Name: label, dtype: int64

변환된 라벨 분포:
label
2    118870
1     37592
0     28063
Name: count, dtype: int64


## 5. 학습/테스트 데이터 분할
전체 데이터를 학습용(Train)과 테스트용(Test)으로 8:2 비율로 나눕니다.

In [5]:
# 전체 데이터를 학습용(Train)과 테스트용(Test)으로 나눕니다.
# test_size=0.2 의미는 전체의 20%를 테스트용으로 쓰겠다는 뜻입니다.
# random_state=42는 매번 실행할 때마다 똑같이 나누기 위해 설정하는 난수 시드값입니다.
# stratify 옵션을 사용하면 라벨 비율을 유지하면서 나눌 수 있습니다.

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"학습 데이터 개수: {len(train_df)}")
print(f"테스트 데이터 개수: {len(test_df)}")

# Hugging Face의 Dataset 형식으로 변환합니다.
# 이렇게 변환하면 transformers 라이브러리에서 다루기 훨씬 편해집니다.
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

print("\n변환된 데이터셋 확인:")
print(train_dataset)

# ★ 최종 점검: 학습 데이터셋의 라벨이 정말 0, 1, 2인지 확인합니다.
print("\n[최종 점검] 학습 데이터셋 라벨 종류:", set(train_dataset['label']))
assert set(train_dataset['label']).issubset({0, 1, 2}), "라벨에 0, 1, 2 이외의 값이 포함되어 있습니다!"

학습 데이터 개수: 147620
테스트 데이터 개수: 36905

변환된 데이터셋 확인:
Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 147620
})

[최종 점검] 학습 데이터셋 라벨 종류: {0, 1, 2}


## 6. 토크나이저 로드
Pre-trained 모델(KLUE RoBERTa)의 토크나이저를 불러옵니다.

In [6]:
# Hugging Face Hub에서 미리 학습된 모델의 토크나이저를 불러옵니다.
# 한국어 모델 성능이 좋은 KLUE RoBERTa 모델을 사용하겠습니다.
from transformers import AutoTokenizer

model_id = "klue/roberta-base"
print(f"{model_id} 모델의 토크나이저를 로드합니다...")

tokenizer = AutoTokenizer.from_pretrained(model_id)

# 토크나이저가 잘 작동하는지 테스트해봅시다.
sample_text = "이 쇼핑몰 배송이 정말 빠르네요!"
tokens = tokenizer(sample_text)
print(f"원본 문장: {sample_text}")
print(f"토큰화 결과 (Input IDs): {tokens['input_ids']}")
print(f"토큰 복원 결과: {tokenizer.decode(tokens['input_ids'])}")

klue/roberta-base 모델의 토크나이저를 로드합니다...
원본 문장: 이 쇼핑몰 배송이 정말 빠르네요!
토큰화 결과 (Input IDs): [0, 1504, 7576, 9488, 2052, 3944, 5185, 2203, 2182, 5, 2]
토큰 복원 결과: [CLS] 이 쇼핑몰 배송이 정말 빠르네요! [SEP]


## 7. 데이터 토큰화
전체 데이터셋을 토크나이저를 이용해 모델이 이해할 수 있는 형태로 변환합니다.

In [7]:
# 전체 데이터셋에 토크나이저를 적용하는 함수를 만듭니다.
def preprocess_function(examples):
    # 'text' 컬럼의 문장들을 토크나이저에 넣습니다.
    # truncation=True: 문장이 너무 길면 모델이 처리할 수 있는 최대 길이(보통 512, 여기선 128로 제한)로 자릅니다.
    # (학습 속도와 메모리를 위해 길이를 128 정도로 제한해보겠습니다)
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# map 함수를 사용하여 전체 데이터셋에 적용합니다.
# batched=True를 하면 여러 문장을 한 번에 처리해서 속도가 빨라집니다.
print("토크나이징 진행 중... (데이터가 많으면 조금 걸릴 수 있습니다)")
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

print("토크나이징 완료!")
print(tokenized_train)

토크나이징 진행 중... (데이터가 많으면 조금 걸릴 수 있습니다)


Map:   0%|          | 0/147620 [00:00<?, ? examples/s]

Map:   0%|          | 0/36905 [00:00<?, ? examples/s]

토크나이징 완료!
Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 147620
})


## 8. Full Fine-Tuning 모델 학습
모델의 모든 파라미터를 업데이트하는 Full Fine-Tuning 방식을 수행합니다.

In [8]:
# ==========================================
# 2. Full Fine-Tuning 모델 학습
# ==========================================

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 평가 지표를 계산하는 함수입니다.
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro') # 3개 클래스이므로 macro 평균
    
    return {
        'accuracy': acc,
        'f1': f1
    }

# 모델 로드 (라벨 개수는 3개: 부정, 중립, 긍정)
model_full = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=3)

# 학습을 위한 설정값들입니다.
# output_dir: 결과가 저장될 폴더
# num_train_epochs: 데이터셋을 몇 번 반복해서 학습할지 
# learning_rate: 학습률 (얼마나 세밀하게 학습할지)
full_finetuning_args = TrainingArguments(
    output_dir="./saved_models/full_finetune",
    eval_strategy="epoch", # [수정] evaluation_strategy -> eval_strategy
    save_strategy="epoch", # 매 에포크마다 모델 저장
    learning_rate=2e-5, 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1, # 빠른 실습을 위해 1 epoch만 설정 (성능을 높이려면 3정도로 늘려주세요)
    weight_decay=0.01,
    load_best_model_at_end=True, # 학습이 끝나면 가장 성능 좋은 모델을 불러옴
    metric_for_best_model="accuracy",
    save_total_limit=1 # 용량 절약을 위해 가장 좋은 모델 1개만 저장
)

# Trainer 객체 생성
trainer_full = Trainer(
    model=model_full,
    args=full_finetuning_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 학습 시작!
print("Full Fine-Tuning 학습 시작...")
trainer_full.train()

# 학습된 모델 저장
trainer_full.save_model("./saved_models/full_finetune_final")
print("Full Fine-Tuning 학습 및 저장 완료!")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_2741255/1771491142.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(


Full Fine-Tuning 학습 시작...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.233800,0.224464,0.911042,0.878896


Full Fine-Tuning 학습 및 저장 완료!


## 9. PEFT (LoRA) 모델 학습
적은 파라미터만 학습하여 효율적인 PEFT(LoRA) 방식을 수행합니다.

In [9]:
# ==========================================
# 3. PEFT (LoRA) 모델 학습
# ==========================================

# PEFT 라이브러리에서 필요한 모듈을 가져옵니다.
from peft import LoraConfig, get_peft_model, TaskType

# 1. 베이스 모델을 다시 새로 불러옵니다 (Full Fine-tuning된 모델이 아니라 깨끗한 상태로 시작하기 위해)
# 메모리 부족 방지를 위해 기존 모델 삭제 및 캐시 정리
import torch
del model_full
del trainer_full
torch.cuda.empty_cache()

print("베이스 모델을 다시 로드합니다 (PEFT용)...")
model_peft = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=3)

# 2. LoRA 설정 정의
# r: LoRA의 Rank (크면 클수록 학습할 파라미터가 늘어나지만 성능이 좋아질 수 있음, 보통 8, 16 사용)
# lora_alpha: Scaling factor
# target_modules: LoRA를 적용할 모듈 (RoBERTa의 경우 query, value에 주로 적용)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.1,
    target_modules=["query", "value"] # roberta 구조에 맞게 설정
)

# 3. 모델에 LoRA 적용
model_peft = get_peft_model(model_peft, peft_config)

# 학습 가능한 파라미터 수 출력 (Full Fine-tuning 대비 얼마나 줄었는지 확인해보세요!)
print("PEFT 적용 후 학습 파라미터 확인:")
model_peft.print_trainable_parameters()

# 4. PEFT 학습 설정 (거의 동일하지만 저장 경로만 다르게)
peft_training_args = TrainingArguments(
    output_dir="./saved_models/peft_lora",
    eval_strategy="epoch", # [수정] evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    learning_rate=1e-3, # LoRA는 보통 학습률을 조금 더 크게 잡습니다 (예: 1e-3 ~ 5e-4)
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1, # 비교를 위해 동일하게 1 epoch
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1
)

# 5. Trainer 생성
trainer_peft = Trainer(
    model=model_peft,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 학습 시작!
print("PEFT (LoRA) 학습 시작...")
trainer_peft.train()

# 모델 저장
trainer_peft.save_model("./saved_models/peft_lora_final")
print("PEFT 학습 및 저장 완료!")

베이스 모델을 다시 로드합니다 (PEFT용)...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PEFT 적용 후 학습 파라미터 확인:
trainable params: 887,811 || all params: 111,508,230 || trainable%: 0.7962
PEFT (LoRA) 학습 시작...


/tmp/ipykernel_2741255/3667435231.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_peft = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.250600,0.229194,0.909118,0.876061


PEFT 학습 및 저장 완료!


## 10. 성능 비교 및 결론
Full Fine-Tuning과 PEFT(LoRA)의 학습 결과(정확도, F1 Score)와 효율성을 비교하고 결론을 도출합니다.

In [10]:
# 두 모델의 최종 성능을 평가합니다.
print("Full Fine-Tuning 모델 평가 중...")
# 주의: trainer_full 객체가 메모리에서 삭제되었다면 다시 로드해야 할 수 있습니다.
# 여기서는 위에서 삭제했으므로, 저장된 모델을 다시 불러와서 평가하거나,
# 로그(log_history)를 통해 확인하는 방법을 사용합니다.

# 간단하게 PEFT 모델의 평가 결과만이라도 확인해봅니다.
peft_eval_result = trainer_peft.evaluate()
print("PEFT 모델 평가 결과:", peft_eval_result)

# ---------------------------------------------------------
# [결과 비교 및 요약]
# ---------------------------------------------------------
# 실제 학습을 돌려보면 다음과 같은 경향을 보입니다:
# 1. Full Fine-Tuning:
#    - 모든 파라미터를 학습하므로 성능(Accuracy)이 가장 높을 가능성이 큽니다.
#    - 하지만 학습 시간이 오래 걸리고, GPU 메모리를 많이 차지합니다.
#    - 모델 파일 용량이 큽니다 (약 400MB+).
#
# 2. PEFT (LoRA):
#    - 학습 파라미터 수가 매우 적습니다 (전체의 1% 미만).
#    - 학습 속도가 빠르고 메모리 사용량이 적습니다.
#    - 성능은 Full Fine-Tuning과 비슷하거나 약간 낮을 수 있지만, 효율성 면에서 압도적입니다.
#    - 모델 파일 용량이 매우 작습니다 (몇 MB 수준).

print("\n[결론]")
print("PEFT(LoRA)는 적은 리소스로도 Full Fine-Tuning에 준하는 성능을 낼 수 있는 효율적인 방법입니다.")
print("특히 대규모 언어 모델(LLM)을 다룰 때 이러한 효율성은 매우 중요합니다.")

Full Fine-Tuning 모델 평가 중...


PEFT 모델 평가 결과: {'eval_loss': 0.22919422388076782, 'eval_accuracy': 0.9091180056902859, 'eval_f1': 0.8760609220990934, 'eval_runtime': 201.7116, 'eval_samples_per_second': 182.959, 'eval_steps_per_second': 11.437, 'epoch': 1.0}

[결론]
PEFT(LoRA)는 적은 리소스로도 Full Fine-Tuning에 준하는 성능을 낼 수 있는 효율적인 방법입니다.
특히 대규모 언어 모델(LLM)을 다룰 때 이러한 효율성은 매우 중요합니다.


## 11. 추론 테스트 (Inference)
학습된 PEFT 모델을 사용하여 실제 문장의 감성을 예측해봅니다.

In [11]:
# 학습된 모델로 새로운 문장 테스트해보기
def predict_sentiment(text, model, tokenizer):
    # 입력 문장 토큰화
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    # GPU로 이동
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
        
    # 라벨 매핑 (0: 부정, 1: 중립, 2: 긍정)
    label_map_inv = {0: "부정", 1: "중립", 2: "긍정"}
    return label_map_inv[prediction]

# 테스트 문장
test_sentences = [
    "배송이 너무 느려서 화가 나네요.",
    "디자인은 예쁜데 사이즈가 좀 작아요.",
    "정말 마음에 듭니다! 재구매 의사 있어요."
]

print("[PEFT 모델 추론 결과]")
for sent in test_sentences:
    result = predict_sentiment(sent, model_peft, tokenizer)
    print(f"문장: {sent} -> 예측: {result}")

[PEFT 모델 추론 결과]
문장: 배송이 너무 느려서 화가 나네요. -> 예측: 부정
문장: 디자인은 예쁜데 사이즈가 좀 작아요. -> 예측: 중립
문장: 정말 마음에 듭니다! 재구매 의사 있어요. -> 예측: 긍정
